# 03 — Exploratory Data Analysis & PCA

This notebook performs exploratory data analysis and Principal Component Analysis (PCA) on the engineered features. We will:
1. Load the feature matrix from notebook 02
2. Analyze feature distributions and identify outliers
3. Examine correlation structure
4. Apply PCA to reduce dimensionality
5. Analyze explained variance
6. Interpret PCA loadings
7. Visualize stocks in PCA space by sector
8. Summarize key findings and save PCA-transformed data

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pca_analysis import StockPCA
from feature_engineering import create_feature_matrix

# Display and plot settings
pd.set_option('display.max_columns', 100)
sns.set_style('whitegrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load Feature Matrix

In [ ]:
# Load feature matrix from previous notebook
try:
    feature_matrix = pd.read_parquet('../data/processed/feature_matrix.parquet')
    feature_metadata = pd.read_csv('../data/processed/feature_metadata.csv')
    print("Feature matrix loaded successfully")
except Exception as e:
    print(f"Error loading feature matrix: {e}")
    print("Please run notebooks 01 and 02 first.")

print(f"\nFeature matrix shape: {feature_matrix.shape}")
print(f"Tickers: {feature_matrix.index.tolist()[:10]}...")
print(f"\nFeature matrix info:")
print(feature_matrix.info())

## 2. Feature Distributions

In [ ]:
# Select numeric features
numeric_features = feature_matrix.select_dtypes(include=[np.number])

# Plot distributions of a few key features
key_features = numeric_features.columns[:8]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for idx, feature in enumerate(key_features):
    data = numeric_features[feature].dropna()
    axes[idx].hist(data, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[idx].set_title(f'{feature}', fontweight='bold')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(alpha=0.3)
    
    mean_val = data.mean()
    std_val = data.std()
    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(mean_val - std_val, color='orange', linestyle=':', linewidth=1, label=f'±1 Std')
    axes[idx].axvline(mean_val + std_val, color='orange', linestyle=':', linewidth=1)
    axes[idx].legend(fontsize=8)

plt.tight_layout()
plt.show()

# Check for outliers
print(f"\nOutlier Detection (using IQR method):")
for feature in key_features:
    Q1 = numeric_features[feature].quantile(0.25)
    Q3 = numeric_features[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_count = ((numeric_features[feature] < lower_bound) | (numeric_features[feature] > upper_bound)).sum()
    if outlier_count > 0:
        print(f"{feature}: {outlier_count} outliers")

## 3. Correlation Analysis

In [ ]:
# Compute correlation matrix
correlation_matrix = numeric_features.corr()

# Identify highly correlated feature pairs
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.85:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

# Sort by absolute correlation
high_corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

print(f"Highly Correlated Feature Pairs (|corr| > 0.85):")
for feat1, feat2, corr in high_corr_pairs[:15]:
    print(f"  {feat1} <-> {feat2}: {corr:.3f}")

# Plot correlation heatmap for a sample of features
sample_features = numeric_features.columns[:15]
sample_corr = correlation_matrix.loc[sample_features, sample_features]

plt.figure(figsize=(12, 10))
sns.heatmap(sample_corr, cmap='RdBu_r', center=0, vmin=-1, vmax=1,
            square=True, annot=True, fmt='.2f', cbar_kws={'label': 'Correlation'})
plt.title('Feature Correlation Matrix (Sample)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Apply PCA

In [ ]:
# Initialize PCA with variance threshold
stock_pca = StockPCA(variance_threshold=0.90)

# Fit and transform the feature matrix
pca_data = stock_pca.fit_transform(numeric_features)

print(f"PCA Analysis Results:")
print(f"  Original features: {numeric_features.shape[1]}")
print(f"  PCA components: {pca_data.shape[1]}")
print(f"  Total variance explained: {stock_pca.explained_variance_ratio_.sum():.4f}")
print(f"\nExplained variance by component:")
for i, var in enumerate(stock_pca.explained_variance_ratio_[:10]):
    cumsum = stock_pca.explained_variance_ratio_[:i+1].sum()
    print(f"  PC{i+1}: {var:.4f} (Cumulative: {cumsum:.4f})")

## 5. Explained Variance Analysis

In [ ]:
# Plot explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].bar(range(1, min(len(stock_pca.explained_variance_ratio_)+1, 21)), 
            stock_pca.explained_variance_ratio_[:20], alpha=0.7, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Principal Component', fontweight='bold')
axes[0].set_ylabel('Explained Variance Ratio', fontweight='bold')
axes[0].set_title('Scree Plot (Top 20 Components)', fontweight='bold', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)

# Cumulative explained variance
cumsum_var = np.cumsum(stock_pca.explained_variance_ratio_)
axes[1].plot(range(1, min(len(cumsum_var)+1, 31)), cumsum_var[:30], 
            marker='o', linewidth=2, markersize=6, color='darkgreen')
axes[1].axhline(y=0.90, color='red', linestyle='--', linewidth=2, label='90% Variance')
axes[1].set_xlabel('Number of Components', fontweight='bold')
axes[1].set_ylabel('Cumulative Explained Variance', fontweight='bold')
axes[1].set_title('Cumulative Explained Variance', fontweight='bold', fontsize=12)
axes[1].grid(alpha=0.3)
axes[1].legend()
axes[1].set_ylim([0, 1.05])

plt.tight_layout()
plt.show()

## 6. PCA Loadings Analysis

In [ ]:
# Get loadings
loadings = pd.DataFrame(
    stock_pca.components_.T,
    columns=[f'PC{i+1}' for i in range(stock_pca.components_.shape[0])],
    index=numeric_features.columns
)

# Display top features for first 3 principal components
for pc_num in range(1, 4):
    pc_name = f'PC{pc_num}'
    top_positive = loadings[pc_name].nlargest(5)
    top_negative = loadings[pc_name].nsmallest(5)
    
    print(f"\n{pc_name} Top Features (Explained Variance: {stock_pca.explained_variance_ratio_[pc_num-1]:.4f}):")
    print(f"  Positive:")
    for feat, val in top_positive.items():
        print(f"    {feat}: {val:.4f}")
    print(f"  Negative:")
    for feat, val in top_negative.items():
        print(f"    {feat}: {val:.4f}")

# Plot loadings heatmap for first 5 components
top_n_features = 20
top_var_features = numeric_features.var().nlargest(top_n_features).index
top_loadings = loadings.loc[top_var_features, ['PC1', 'PC2', 'PC3', 'PC4', 'PC5']]

plt.figure(figsize=(10, 12))
sns.heatmap(top_loadings, cmap='RdBu_r', center=0, annot=True, fmt='.2f',
            cbar_kws={'label': 'Loading'}, linewidths=0.5)
plt.title('PCA Loadings (Top 20 Features by Variance)', fontweight='bold', fontsize=12)
plt.tight_layout()
plt.show()

## 7. Stocks in PCA Space

In [ ]:
# Create a sector mapping (example)
sector_mapping = {
    'AAPL': 'Technology', 'MSFT': 'Technology', 'GOOGL': 'Technology', 'AMZN': 'Consumer',
    'NVDA': 'Technology', 'META': 'Technology', 'AMD': 'Technology', 'INTC': 'Technology',
    'TSLA': 'Consumer', 'CRM': 'Technology', 'NFLX': 'Consumer', 'DIS': 'Consumer',
    'JPM': 'Financials', 'GS': 'Financials', 'BAC': 'Financials', 'WFC': 'Financials',
    'V': 'Financials', 'MA': 'Financials',
    'JNJ': 'Healthcare', 'UNH': 'Healthcare', 'LLY': 'Healthcare', 'ABBV': 'Healthcare',
    'MRK': 'Healthcare', 'PFE': 'Healthcare',
    'XOM': 'Energy', 'CVX': 'Energy', 'COP': 'Energy', 'MPC': 'Energy',
    'PG': 'Consumer', 'KO': 'Consumer', 'PEP': 'Consumer', 'WMT': 'Consumer',
    'HD': 'Consumer',
    'BA': 'Industrials', 'GE': 'Industrials', 'CAT': 'Industrials', 'AVGO': 'Technology'
}

# Get sector for each ticker in pca_data
sectors = pd.Series(
    [sector_mapping.get(ticker, 'Other') for ticker in pca_data.index],
    index=pca_data.index
)

# Create 2D scatter plot of stocks in PCA space
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# PC1 vs PC2
for sector in sectors.unique():
    mask = sectors == sector
    axes[0].scatter(pca_data[mask, 0], pca_data[mask, 1], 
                   s=150, alpha=0.7, label=sector, edgecolors='black', linewidth=1.5)

axes[0].set_xlabel(f'PC1 ({stock_pca.explained_variance_ratio_[0]:.2%} variance)', fontweight='bold')
axes[0].set_ylabel(f'PC2 ({stock_pca.explained_variance_ratio_[1]:.2%} variance)', fontweight='bold')
axes[0].set_title('Stocks in PCA Space (PC1 vs PC2)', fontweight='bold', fontsize=12)
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)
axes[0].axvline(x=0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)

# Add ticker labels
for i, ticker in enumerate(pca_data.index):
    axes[0].annotate(ticker, (pca_data[i, 0], pca_data[i, 1]), 
                    fontsize=8, ha='center', va='center')

# PC1 vs PC3
for sector in sectors.unique():
    mask = sectors == sector
    axes[1].scatter(pca_data[mask, 0], pca_data[mask, 2], 
                   s=150, alpha=0.7, label=sector, edgecolors='black', linewidth=1.5)

axes[1].set_xlabel(f'PC1 ({stock_pca.explained_variance_ratio_[0]:.2%} variance)', fontweight='bold')
axes[1].set_ylabel(f'PC3 ({stock_pca.explained_variance_ratio_[2]:.2%} variance)', fontweight='bold')
axes[1].set_title('Stocks in PCA Space (PC1 vs PC3)', fontweight='bold', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)
axes[1].axvline(x=0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)

# Add ticker labels
for i, ticker in enumerate(pca_data.index):
    axes[1].annotate(ticker, (pca_data[i, 0], pca_data[i, 2]), 
                    fontsize=8, ha='center', va='center')

plt.tight_layout()
plt.show()

## 8. Key Findings

In [ ]:
# Summary of key findings
findings = f"""
=== PCA ANALYSIS SUMMARY ===

Dimensionality Reduction:
  Original feature dimensions: {numeric_features.shape[1]}
  Reduced to {pca_data.shape[1]} principal components
  Reduction ratio: {(1 - pca_data.shape[1]/numeric_features.shape[1])*100:.1f}%
  Variance retained: {stock_pca.explained_variance_ratio_.sum():.2%}

Component Importance:
  PC1: {stock_pca.explained_variance_ratio_[0]:.2%} of variance
  PC2: {stock_pca.explained_variance_ratio_[1]:.2%} of variance
  PC3: {stock_pca.explained_variance_ratio_[2]:.2%} of variance
  First 5 PCs: {stock_pca.explained_variance_ratio_[:5].sum():.2%} cumulative variance

Feature Correlations:
  Highly correlated pairs (|r| > 0.85): {len(high_corr_pairs)}
  This multicollinearity is handled well by PCA

Sector Observations:
  Technology stocks show {len(sectors[sectors == 'Technology'])} representatives
  Financials stocks show {len(sectors[sectors == 'Financials'])} representatives
  Healthcare stocks show {len(sectors[sectors == 'Healthcare'])} representatives

Next Steps:
  Use PCA-transformed data for clustering analysis
  Apply machine learning models with reduced feature set
  Validate predictions using original features as well
"""

print(findings)

In [ ]:
import os

# Save PCA-transformed data
os.makedirs('../data/pca', exist_ok=True)

# Create DataFrame with PCA components
pca_df = pd.DataFrame(
    pca_data,
    columns=[f'PC{i+1}' for i in range(pca_data.shape[1])],
    index=numeric_features.index
)

# Save PCA results
pca_df.to_parquet('../data/pca/pca_transformed_data.parquet')
loadings.to_parquet('../data/pca/pca_loadings.parquet')

# Save variance explained
variance_df = pd.DataFrame({
    'component': [f'PC{i+1}' for i in range(len(stock_pca.explained_variance_ratio_))],
    'explained_variance': stock_pca.explained_variance_ratio_,
    'cumulative_variance': np.cumsum(stock_pca.explained_variance_ratio_)
})
variance_df.to_csv('../data/pca/explained_variance.csv', index=False)

print("PCA results saved successfully!")
print(f"  - pca_transformed_data.parquet ({pca_df.shape})")
print(f"  - pca_loadings.parquet ({loadings.shape})")
print(f"  - explained_variance.csv")